## BatRaVot Protocol
The goal of this scheme is to allow for off-chain voting for Ethereum-based DAOs (Decentralized Autonomous Organisations), with on-chain verification. The goal is to reduce the cost of voting, which can be quite high if the voting was done directly on the Ethereum blockchain.

BatRaVot is a binary voting scheme, which means that voters can only choose between two options.

### Public Parameter Generation 
We use SHA256 as a hashing function and BLS12-381 as our EC and Tate Pairing as a biliniar mapping.

In [1]:
# Loading the Pairing from external file 
# Note that instead of using a bi-linear pairing such as Weil Pairing we use 
load("bls-pairing.sage")

# Hashing algo
from hashlib import sha256

# Hash the message with SHA256, return an integer 
def hash256(m):
    
    hex_hash = sha256(str(m).encode("utf-8")).hexdigest()
    
    return int(hex_hash, 16)

# Calculate a hash of a message that is in G1
def hashing_algo(m):
    hashI = hash256(m) # An integer hash of a message
    
    hashF = F1(hashI)
    
    while True:
        
        hashG1P = G1 * hashF # This is an alternitive way by mutliplying G2 by the hashF - point in G2 on E2
        
        if hashG1P != 0:
            break
            
        hashF += 1
        
    return hashG1P 

### Election Setup

In [2]:
import random
random.seed(0)
# A unique public election identifier 
# We take  256 bits as this is enough randomness 
election_id = random.getrandbits(256)
# Generate vote specifiers
gs = [hashing_algo(str(election_id) + '0'), hashing_algo(str(election_id) + '1')]
# Note that gs[0] = g0, gs[1] = g1 as in the paper


### Voters

In [3]:
class Voter:
    def __init__(self):
        self.prk = G1F.random_element() # element of G1F
        self.pbk = self.prk * G1 # Point of E1 generated by G1 
        
    def get_pbk(self):
        return self.pbk
        
    # Agree represents if the voter agrees or not
    def vote(self, agrees):
    
        # cast agree to a boolean 
        agrees = bool(agrees)
        
        # This is a valid balot. 
        # If we vote 'Yes' it will select g1
        # If we vote 'No' it will select g0
        # In a balot we additionally return if the voters agrees or not
        # And the voters public key
        return self.prk * gs[agrees], agrees, self.pbk
  
amount_of_voters = 20
voters = [Voter() for i in range(amount_of_voters)]

In [4]:
class Verifier:
    @staticmethod
    def verify_balot(balot, agrees, pbk):
        lhs = BLSpairing.tate_pair(balot, G1)
        rhs = BLSpairing.tate_pair(gs[agrees], pbk)
        assert (lhs == rhs, "Vote count is invalid.")
        
    @staticmethod
    def verify_election_proof(proof, keys):

        keys_with_vote = list(zip(keys, proof[1]))
        
        # Keys that we are told have agreed 
        K1 = list(map(lambda x: x[0], filter(lambda x: x[1], keys_with_vote)))
        if (len(K1) == 0): # If no votes in favour, we can replace by generator, which would act as one
            H1 = G1
        elif (len(K1) == 1): # If one vote in favour, we do not need to add up, just put it
            H1 = K1[0]
        else:
            H1 = reduce(lambda x, y: x + y, K1)
        # Keys that we are told have not agreed 
        K0 = list(map(lambda x: x[0], filter(lambda x: not(x[1]), keys_with_vote)))
        if (len(K0) == 0): # If all votes in favour, we can replace by generator, which would act as one
            H0 = G1
        elif (len(K0) == 1): # If one vote against, we do not need to add up, just put it
            H0 = K0[0]
        else:
            H0 = reduce(lambda x, y: x + y, K0)
        
        
        lhs = BLSpairing.tate_pair(proof[0], G1)
        rhs = BLSpairing.tate_pair(H0, gs[0]) * BLSpairing.tate_pair(H1, gs[1])
        
        assert (lhs == rhs, "Election proof is invalid.")

In [5]:
# Example of voting for True as a first voter and verifying it
balot = voters[0].vote(True)
# Example of verifying the vote of the first voter:
Verifier.verify_balot(balot[0], balot[1], balot[2])
print("Balot prooved successfully!")

Balot prooved successfully!


In [6]:
class Proover:
    def generate_proof(balots):
        # Add together all balots to aggregate a proof
        # And make prooving result correctness easier
        gamma = reduce(lambda x, y: x+y, map(lambda x: x[0], balots))
        # Instead of using I0 and I1 we have an I 
        # where elements are how the voter has voted
        I = list(map(lambda x: x[1], balots))
        return (gamma, I)

In [7]:
# Example of holding an election and verifying it
balots = list(map(lambda voter: voter.vote(random.choice([True, False])), voters))
print("Amount of balots for is {} / {}".format(sum(map(lambda x: x[1], balots)), len(balots)) )
# Proover generates a proof for the balots it knows
proof = Proover.generate_proof(balots)
pbks = list(map(lambda balot: balot[2], balots))
# Verify the election
Verifier.verify_election_proof(proof, pbks)
print("Election result prooved successfully!")

Amount of balots for is 8 / 20
Election result prooved successfully!
